In [1]:
pip install nnfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import nnfs     
from nnfs.datasets import spiral_data
from nnfs.datasets import vertical_data
import matplotlib.pyplot as plt

In [3]:
nnfs.init()   #initialize get random value #geneartes same value every time

class Layer_Dense:
  def __init__(self, n_input, n_neurons):
    self.weights = 0.10 * np.random.rand(n_input, n_neurons)
    self.biases = np.zeros((1, n_neurons), dtype=float, order='C')
    
  def forward(self, inputs):
    self.output = np.dot(inputs, self.weights) + self.biases


In [4]:
X, y = vertical_data(samples=100, classes=3)    #X = 2 features , y = 3 labels. 

In [6]:
dense = Layer_Dense(2, 3) # first dense layer, 2 input

In [7]:
dense.weights

array([[0.03917969, 0.02421786, 0.02503982],
       [0.04833935, 0.00399928, 0.06397051]])

In [8]:
dense.biases

array([[0., 0., 0.]])

In [9]:
dense.forward(X)

In [ ]:
dense.output

In [11]:
class Activation_ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

In [12]:
activation = Activation_ReLU()

In [13]:
activation.forward(dense.output)

In [ ]:
activation.output

In [15]:
class Activation_Softmax:
    def forward(self, inputs):
        exp_val = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        prob = exp_val / np.sum(exp_val, axis=1, keepdims=True)
        self.output = prob

In [16]:
act = Activation_Softmax()
act.forward(activation.output)

In [ ]:
act.output

In [ ]:
# Create model
dense1 = Layer_Dense(2, 3) # first dense layer, 2 inputs
activation1 = Activation_ReLU()

dense2 = Layer_Dense(3, 3) # first dense layer, 2 inputs
activation2 = Activation_ReLU()

dense3 = Layer_Dense(3, 3) # first dense layer, 2 inputs
activation3 = Activation_Softmax()

dense1.forward(X)
activation1.forward(dense1.output)

dense2.forward(activation1.output)
activation2.forward(dense2.output)

dense3.forward(activation2.output)
activation3.forward(dense3.output)

activation3.output

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='brg')
plt.show()

In [ ]:
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss

class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)

        if len(y_true.shape) == 1:
            correct_conf = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            correct_conf = np.sum(y_pred_clipped * y_true, axis=1)
        neg_log_lhd = -np.log(correct_conf)
        return neg_log_lhd

In [ ]:
# Create model
activation1 = Activation_ReLU()
dense2 = Layer_Dense(3, 3) # second dense layer, 3 inputs, 3 outputs
activation2 = Activation_Softmax()
# Create loss function
loss_function = Loss_CategoricalCrossentropy()
dense1.output()


print(dense1.weights)

# Helper variables
lowest_loss = 9999999 # some initial value
best_dense1_weights = dense1.weights.copy()
best_dense1_biases = dense1.biases.copy()
best_dense2_weights = dense2.weights.copy()
best_dense2_biases = dense2.biases.copy()

for iteration in range(10000):
    # Generate a new set of weights for iteration
    dense1.weights += 0.05 * np.random.randn(2, 3)
    dense1.biases += 0.05 * np.random.randn(1, 3)
    dense2.weights += 0.05 * np.random.randn(3, 3)
    dense2.biases += 0.05 * np.random.randn(1, 3)

    # Perform a forward pass of the training data through this layer
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)

    # Perform a forward pass through activation function
    # it takes the output of second dense layer here and returns loss
    loss = loss_function.calculate(activation2.output, y)

    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(activation2.output, axis=1)
    accuracy = np.mean(predictions==y)
    # If loss is smaller - print and save weights and biases aside
    if loss < lowest_loss:
        print('New set of weights found, iteration:', iteration, 'loss:', loss, 'acc:', accuracy)
        best_dense1_weights = dense1.weights.copy()
        best_dense1_biases = dense1.biases.copy()
        best_dense2_weights = dense2.weights.copy()
        best_dense2_biases = dense2.biases.copy()
        lowest_loss = loss
        # Revert weights and biases
    else:
        dense1.weights = best_dense1_weights.copy()
        dense1.biases = best_dense1_biases.copy()
        dense2.weights = best_dense2_weights.copy()
        dense2.biases = best_dense2_biases.copy()